In [3]:
pip install tensorflow_datasets

     |████████████████████████████████| 3.9 MB 637 kB/s eta 0:00:01
     |████████████████████████████████| 48 kB 7.9 MB/s  eta 0:00:01
     |████████████████████████████████| 86 kB 8.9 MB/s  eta 0:00:01
     |████████████████████████████████| 129 kB 23.9 MB/s eta 0:00:01
     |████████████████████████████████| 198 kB 21.4 MB/s eta 0:00:01
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21494 sha256=3e5276dd79093dc4506b2a730245557197c1753346f168741eb93c8b5439f70c
  Stored in directory: /Users/himanibali/Library/Caches/pip/wheels/54/aa/01/724885182f93150035a2a91bce34a12877e8067a97baaf5dc8
Successfully built promise
  Attempting uninstall: absl-py
    Found existing installation: absl-py 0.13.0
    Uninstalling absl-py-0.13.0:
      Successfully uninstalled absl-py-0.13.0
Note: you may need to restart the kernel to use updated packages.


In [5]:
import numpy as np
import tensorflow as tf 

import tensorflow_datasets as tfds

# Data 

In [15]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info = True, as_supervised = True)

mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples

num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image,label): 
    image = tf.cast(image, tf.float32)
    image/= 255.
    return image, label 

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))



# Outline the model 

In [19]:
input_size = 784
output_size = 10
hidden_layer_size = 100 

model = tf.keras.Sequential([ 
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                            tf.keras.layers.Dense(output_size, activation = 'softmax')
                           ])

# Choose the optimizer and the loss function 

In [20]:
model.compile(optimizer='adam',loss = 'sparse_categorical_crossentropy',metrics = ['accuracy'])



# Training 

In [21]:
NUM_EPOCHS = 5
model.fit(train_data, epochs = NUM_EPOCHS, validation_data=(validation_inputs,validation_targets), verbose = 2) 

Epoch 1/5
540/540 - 2s - loss: 0.3391 - accuracy: 0.9046 - val_loss: 0.1597 - val_accuracy: 0.9552
Epoch 2/5
540/540 - 2s - loss: 0.1405 - accuracy: 0.9578 - val_loss: 0.1238 - val_accuracy: 0.9647
Epoch 3/5
540/540 - 2s - loss: 0.0989 - accuracy: 0.9706 - val_loss: 0.0855 - val_accuracy: 0.9742
Epoch 4/5
540/540 - 2s - loss: 0.0746 - accuracy: 0.9771 - val_loss: 0.0723 - val_accuracy: 0.9783
Epoch 5/5
540/540 - 2s - loss: 0.0597 - accuracy: 0.9815 - val_loss: 0.0647 - val_accuracy: 0.9813


# TEST THE MODEL

In [22]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 1s 516ms/step - loss: 0.0825 - accuracy: 0.9741


In [25]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100))


Test loss: 0.08. Test accuracy: 97.41%
